# Importing Packages Needed

In [1]:
import numpy as np

In [2]:
import scipy as sp

In [3]:
import matplotlib.pyplot as py

In [4]:
from scipy.optimize import curve_fit

In [5]:
from scipy.signal import find_peaks

In [6]:
import collections as col

# Definition of Named Tuples

In [7]:
component = col.namedtuple('component','phase intensity')

In [8]:
pulse_component = col.namedtuple('pulse_component','frequency comp1 comp2')

In [9]:
component_separation = col.namedtuple('component_separation','frequency separation')

# Definition of Functions

Importing Data one pulse at a time

In [10]:
def readsinglepulse(j, line):
    
    single_pulse= np.empty([0,1], dtype=fulldatatype)
    
    while int(line[0])==j:
                        
            single_phase= np.array((float(line.split()[0]),float(line.split()[1]),float(line.split()[2]),float(line.split()[3])), dtype=fulldatatype)  
            #Can use for error checking
            #print(single_phase)
            #print(single_phase['frequency'])
            
            single_pulse=np.row_stack((single_pulse, single_phase))              
            
            #print(single_pulse)
                            
            line = spd.readline()
    
    return single_pulse,line

Gaussian Fitting Functions:

In [11]:
def gauss(x,cen,sd,amp):
    return amp*np.exp(-(x-cen)**2/2/sd**2)

def threegauss(x,cen1,sd1,amp1,cen2,sd2,amp2,cen3,sd3,amp3):
    return gauss(x,cen1,sd1,amp1)+gauss(x,cen2,sd2,amp2)+gauss(x,cen3,sd3,amp3)

In [12]:
def fitting_parameters(x,y,expected):
  params,cov=curve_fit(threegauss,x,y,expected)
  sigma=np.sqrt(np.diag(cov))
  return params, sigma

def gauss_fit(x,params):
  return threegauss(x,*params)

Finding Peak of Fittings Function

In [13]:
def gauss_peaks(fit, nofreqband, freqband):
  status='starting'
  
  i=3
  peakpos = find_peaks(fit, prominence=i, distance=50, height=i)[0]

  while len(peakpos) !=2 and i>=1 and i<5:
    if len(peakpos) < 2:
      i=i-0.5
      peakpos = find_peaks(fit, prominence=i, distance=50, height=1)[0]

    elif len(peakpos) > 2:
      i=i+0.5
      peakpos = find_peaks(fit, prominence=i, distance=50, height=2)[0]
    
  #Can be used for error checking
  #else:
    #print('Loop done')
    #print(i)

  if len(peakpos) == 2:
    status='pending'

    if peakpos[0]>=300 and peakpos[0]<=535 and peakpos[1]>=650 and peakpos[1]<=800:
      status='complete'
    else:
      status='error'

  else:
    status='error'


  if status=='complete':
    component1 = component(phase=(360/1024)*peakpos[0], intensity=fit[peakpos[0]])
    component2 = component(phase=(360/1024)*peakpos[1], intensity=fit[peakpos[1]])
  
  elif status=='error':
    component1 = component(phase=0, intensity=0)
    component2 = component(phase=0, intensity=0)

  else:
    print("Error: Something not complete in peak finding")

  freq=(((500-300)/nofreqband)*(nofreqband-freqband))+300 

  profile_comp = pulse_component(frequency=freq, comp1=component1, comp2=component2)

  #Can be used for error checking
  #if status != 'error':
    #plot_peaks(x,y,profile_comp)

  return profile_comp

Plotting Fittings Functions

In [14]:
def plot_fit(x,y,fit,params,sigma):
  py.plot(x, y, lw=0.5, label='data')
  py.plot(x,fit, color='red',lw=3,label='gaussian fit')
  py.xlabel('Phase Bin')
  py.ylabel('Intensity')
  py.legend()
  return

In [15]:
def plot_fill(x,y,params):
  params1 = params[0:3]
  params2 = params[3:6]
  params3 = params[6:9]
  gfit1 = gauss(x, *params1)
  gfit2 = gauss(x, *params2)
  gfit3 = gauss(x, *params3)


  py.plot(x, y, lw=0.5, label='data')
  py.plot(x,fit,color='red',lw=3,label='gaussian fit')
  py.xlabel('Phase Bin')
  py.ylabel('Intensity')
  py.legend()

  py.plot(x, gfit1, "g")
  py.fill_between(x, gfit1.min(), gfit1, facecolor="green", alpha=0.5)
  
  py.plot(x, gfit2, "y")
  py.fill_between(x, gfit2.min(), gfit2, facecolor="yellow", alpha=0.5)

  py.plot(x, gfit3, "b")
  py.fill_between(x, gfit3.min(), gfit3, facecolor="blue", alpha=0.5)

  return 

In [16]:
def plot_peaks(x,y,profile_comp):
  py.plot(x,y, lw=0.5)
  py.plot(x, fit, color='red', lw=3, label='gaussian fit')

  x=[profile_comp[i][0] for i in range(1,3)]
  y=[profile_comp[i][1] for i in range(1,3)]

  py.plot(x, y ,'X', markerfacecolor='black', markeredgecolor='black', label='peaks')

  py.xlabel('Phase (deg)')
  py.ylabel('Intensity')
  py.legend()
  return

Functions for Fitting Separation Data

In [17]:
def powerlaw(x, A, alpha, smin):
  return (A * (x**(-alpha)) + smin)

# Importing Data

Data must be in text file format from software pdv.
Must be one file containing all pulses.

In [18]:
fulldatatype=([('pulse','i8'),('frequency','i8'),('phase','i8'),('intensity','f8')])

In [21]:
with open('../archivefiles/com_archivefiles/pdv/J0820_s_f28.pdv', 'r') as spd:
    
    #Skim past the hashed first line
    line = spd.readline()
    
    #First read
    line = spd.readline()
    
    #number of pulses
    for j in range(0,17):
             
        single_pulse, line = readsinglepulse(j, line)
        
        #no_freq_bands = max(single_pulse['frequency'])+1
        
        pulse_components = []
        
        expected=(501,30,8.02,655,80,4.07,741,20,11.29)
        
#        for i in range(0,no_freq_bands):
 #           freqband = i
  #
   #         #Can be used for error checking
    #        print(freqband)

    
     #       f = single_pulse['frequency'] == freqband

      #      x=single_pulse['phase'][f]
       #     y=single_pulse['intensity'][f]

        #    if len(x)!=0 and len(y)!=0:
         #       params,sigma = fitting_parameters(x,y,expected)
          #      fit = gauss_fit(x, params)
           #     profile_comp = gauss_peaks(fit, no_freq_bands, freqband)

                #Can be used for error checking
                #print(profile_comp)
  
            #    pulse_components.append(profile_comp)
        

In [22]:
single_pulse[0]

IndexError: index 0 is out of bounds for axis 0 with size 0

## Checking Individual Frequency

In [ ]:
freqband = 89
f = single_pulse['frequency'] == freqband

In [ ]:
phase=single_pulse['phase'][f]
intensity=single_pulse['intensity'][f]

In [ ]:
x = phase
y = intensity

In [ ]:
expected=(501,30,8.02,655,80,4.07,741,20,11.29)
params,sigma = fitting_parameters(x,y,expected)
fit = gauss_fit(x, params)
profile_comp = gauss_peaks(fit,no_freq_bands,freqband)

In [ ]:
plot_fill(x,y,params)

In [ ]:
plot_peaks(x*(360/1024),y,profile_comp)

# Gaussian Plotting and Finding Peaks of Data

The following is a looped version of the previous section to find the correct peaks for each pulse profile

In [ ]:
all_components = []

In [ ]:
#Taken from Peaks and Gaussian fittings of frequency band 73, in the middle
expected=(501,30,8.02,655,80,4.07,741,20,11.29)

for i in range(0,no_freq_bands):
  freqband = i
  
  #Can be used for error checking
  #print(freqband)

  f = averageprofiles['frequency'] == freqband

  x=averageprofiles['phase'][f]
  y=averageprofiles['intensity'][f]

  if len(x)!=0 and len(y)!=0:
    params,sigma = fitting_parameters(x,y,expected)
    fit = gauss_fit(x, params)
    profile_comp = gauss_peaks(fit, no_freq_bands, freqband)

    #Can be used for error checking
    #print(profile_comp)
  
    all_components.append(profile_comp)

# Movement of Peak Position over Frequency

Starting to graph out all the movement of component phase across frequency

---



In [ ]:
len(all_components)

In [ ]:
phase_comp1=[]
frequency_comp1=[]

for i in range(0,len(all_components)):
  if all_components[i][1][0] != 0:
    phase_comp1.append(all_components[i][1][0])
    frequency_comp1.append(all_components[i][0])

In [ ]:
py.plot(phase_comp1,frequency_comp1, '.')
py.xlabel('Phase Bin')
py.ylabel('Frequency')

In [ ]:
phase_comp2=[]
frequency_comp2=[]

for i in range(0,len(all_components)):
  if all_components[i][2][0] != 0:
    phase_comp2.append(all_components[i][2][0])
    frequency_comp2.append(all_components[i][0])

In [ ]:
py.plot(phase_comp2,frequency_comp2, '.')
py.xlabel('Phase Bin')
py.ylabel('Frequency')

# Component Separation Across Frequency

In [ ]:
def separation_singleprofile(components_array):
  sep_array=[]

  for i in range(0,len(components_array)):
    if components_array[i][1][0] != 0 and components_array[i][2][0] != 0:
    
      freq=components_array[i][0]
      sep=(components_array[i][2][0]) - (components_array[i][1][0])

      single_sep=component_separation(frequency=freq, separation=sep)

      sep_array.append(single_sep)

  return sep_array

In [ ]:
component_sep = separation_singleprofile(all_components)

In [ ]:
component_sep

In [ ]:
comp_sep=[]
frequency_forsep=[]

#range(0,no_freq_bands)
for i in range(0,len(component_sep)):
  comp_sep.append(component_sep[i][1])
  frequency_forsep.append(component_sep[i][0])

In [ ]:
py.plot(frequency_forsep, comp_sep, '.')

# Attempted Fitting Power Laws to Separation

In [ ]:
expect=(200,0.5,20)
sep_params,sep_cov=curve_fit(powerlaw,frequency_forsep,comp_sep, maxfev=1000000)

In [ ]:
print(sep_params)
separation_fit = powerlaw(frequency_forsep, *sep_params)

In [ ]:
py.plot(frequency_forsep, comp_sep, lw=0.5, label='data')
py.plot(frequency_forsep,separation_fit, color='red',lw=1,label='power fit')

In [ ]:
chi, p = sp.stats.chisquare(comp_sep, separation_fit)
print(f'The equation of fit for this pulsar is: \t \u0394\u03B8 = {sep_params[0]:.3f} \u03BD^-{sep_params[1]:.3f} + {sep_params[2]:.3f} \n')
print('The chi squared value for the fit is: ', chi)
print('Units for \u0394\u03B8 is degrees and \u03BD is MHz')

# IGNORE - Attempted Fitting Power Laws to Phase

Below is attempts at fitting power laws. Ignore for now.

In [ ]:
def powerlaw(x, A, alpha):
  return A * (x**alpha)

def powerlawint(x, A, alpha, c):
  return ((A*x**(alpha+1))/(alpha+1))+c

In [ ]:
expect=[5.72993437e+15, -4.85937668]
params,cov=curve_fit(powerlaw,xaxis_comp1,yaxis_comp2, maxfev=10000)
#sigma=np.sqrt(np.diag(cov))

In [ ]:
params

In [ ]:
x=range(470,570)

In [ ]:
py.plot(xaxis_comp1,yaxis_comp1, lw=0.5, label='data')
py.plot(x,powerlaw(x, *params), color='red',lw=1,label='power fit')

In [ ]:
print(xaxis_comp1)
print(yaxis_comp1)
print(len(xaxis_comp1))
print(len(yaxis_comp1))

In [ ]:
params,cov=curve_fit(powerlaw,xaxis_comp1_rev,yaxis_comp1_rev, maxfev=10000)
#sigma=np.sqrt(np.diag(cov))

In [ ]:
params

In [ ]:
x=range(290,560)

In [ ]:
py.plot(xaxis_comp1_rev,yaxis_comp1_rev, lw=0.5, label='data')
py.plot(x,powerlaw(x, *params), color='red',lw=1,label='power fit')